# Visitors

The following example shows how to navigate an AST in Python.

In [ ]:
import ast
your_code = """
def foo():  # type: ignore
    ret = 2 * 2
    return ret
"""
your_ast = ast.parse(your_code)
print(your_ast)
print(your_ast.type_ignores)
print(your_ast.body[0])
print(your_ast.body[0].name)

You can refer to the official document (https://docs.python.org/3/library/ast.html) to figure out available attributes for each AST node type. For example,

```
FunctionDef(identifier name, arguments args,
                       stmt* body, expr* decorator_list, expr? returns,
                       string? type_comment, type_param* type_params)
```

However, it is too tedious to explore AST nodes and their attributes one-by-one. Thus, it is necessary to come up with a better navigation method. Fortunately, `ast` package provides visitors to traverse a given AST. You can find an example below:

In [ ]:
from ast import NodeVisitor, FunctionDef

class YourVisitor(NodeVisitor):
    def __init__(self) -> None:
        super().__init__()
    
    def visit_FunctionDef(self, node: FunctionDef):
        print("FunctionDef:", node.name)
            

In [ ]:
avisitor = YourVisitor()
avisitor.visit(your_ast)

Let's try a longer code.

In [ ]:
long_code = """
import pickle

import six


class MetaNode(type):
    def __new__(mcs, name, bases, dict):
        attrs = list(dict['attrs'])
        dict['attrs'] = list()

        for base in bases:
            if hasattr(base, 'attrs'):
                dict['attrs'].extend(base.attrs)

        dict['attrs'].extend(attrs)

        return type.__new__(mcs, name, bases, dict)


@six.add_metaclass(MetaNode)
class Node(object):
    attrs = ()

    def __init__(self, **kwargs):
        values = kwargs.copy()

        for attr_name in self.attrs:
            value = values.pop(attr_name, None)
            setattr(self, attr_name, value)

        if values:
            raise ValueError('Extraneous arguments')

    def __equals__(self, other):
        if type(other) is not type(self):
            return False

        for attr in self.attrs:
            if getattr(other, attr) != getattr(self, attr):
                return False

        return True

    def __repr__(self):
        attr_values = []
        for attr in sorted(self.attrs):
            attr_values.append('%s=%s' % (attr, getattr(self, attr)))
        return '%s(%s)' % (type(self).__name__, ', '.join(attr_values))

    def __iter__(self):
        return walk_tree(self)

    def filter(self, pattern):
        for path, node in self:
            if ((isinstance(pattern, type) and isinstance(node, pattern)) or
                (node == pattern)):
                yield path, node

    @property
    def children(self):
        return [getattr(self, attr_name) for attr_name in self.attrs]
    
    @property
    def position(self):
        if hasattr(self, "_position"):
            return self._position

def walk_tree(root):
    children = None

    if isinstance(root, Node):
        yield (), root
        children = root.children
    else:
        children = root

    for child in children:
        if isinstance(child, (Node, list, tuple)):
            for path, node in walk_tree(child):
                yield (root,) + path, node

def dump(ast, file):
    pickle.dump(ast, file)

def load(file):
    return pickle.load(file)
"""

In [ ]:
long_ast = ast.parse(long_code)
ast.dump(long_ast)

In [ ]:
avisitor.visit(long_ast)

In [ ]:
from ast import Name, Call

class YourVisitor(YourVisitor):
    def visit_Name(self, node: Name):
        print("Name reference:", node.id)
    
    def visit_Call(self, node: Call):
        print("Func Invocation:", node.func, node.args)

In [ ]:
bvisitor = YourVisitor()
bvisitor.visit(long_ast)

In [ ]:
class YourVisitor(YourVisitor):
    def __init__(self) -> None:
        super().__init__()
        
    def visit_FunctionDef(self, node: FunctionDef):
        print("FunctionDef:", node.name)
        return self.generic_visit(node)
    
    def visit_Name(self, node: Name):
        print("Name reference:", node.id)
        return self.generic_visit(node)
    
    def visit_Call(self, node: Call):
        print("Func Invocation:", node.func, node.args)
        return self.generic_visit(node)

In [ ]:
bvisitor = YourVisitor()
bvisitor.visit(long_ast)